<a href="https://colab.research.google.com/github/JoaoVLima/ActualSimplePDFValidator/blob/main/TDE01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PUCPR
## **João Victor de Lima**
### TDE01 - Construção de Interpretadores
**Desenvolver um programa que analise documentos no
formato simplepdf**, uma versão textual simplificada inspirada na estrutura interna de arquivos pdf.
O programa deverá **processar documentos simplepdf**, **validar sua estrutura segundo as regras formais da linguagem que aqui será definida**, **gerar estatísticas** e **transformações do conteúdo**.

[adobe pdf docs (for reference)](https://opensource.adobe.com/dc-acrobat-sdk-docs/pdfstandards/pdfreference1.0.pdf)

# Explicação

## Formato SimplePDF

O SimplePDF é combosto por:

1. **Cabeçalho**: Todo documento começa com uma linha de cabeçalho
```
%SPDF-1.0
```

2. **Objetos**: O documento consiste em objetos numerados no formato
```
obj_id obj_gen obj
<< conteudo >>
endobj
```
No qual:
  - **obj_id** é um número inteiro único que identifica o objeto
  - **obj_gen** é um número de geração
(normalmente 0)

3. **Tipos de dados básicos**:
  - **Números**: inteiros (123) ou decimais (123.45)
  - **Strings**: delimitadas por parênteses: (Este é um texto)
  - **Nomes**: começam com /: /Nome
  - **Booleanos**: true ou false
  - **Null**: null
  - **Referências**: indicam referência a outro objeto: obj_id obj_gen R

4. **Estruturas de dados**:
  - **Dicionários**: pares chave/valor no formato: /Chave valor
  - **Arrays**: sequências de valores delimitadas por [ e ]
  - **Streams**: Blocos de dados entre stream e endstream

5. **Estrutura hierárquica**:
  - **Objeto catálogo (root)**: Define a estrutura do documento
  - **Objeto de páginas**: Mantém lista de páginas
  - **Objetos de página**: Contêm conteúdo e propriedades de cada página
  - **Objetos de conteúdo**: Contêm texto e elementos gráficos


6. **Comentários**:
  Linhas que começam com % (exceto o cabeçalho) são comentários

7. **Cross-reference table (xref)**: Tabela que lista a posição de cada objeto no arquivo:
```
xref
0 3
0000000000 65535 f
0000000010 00000 n
0000000079 00000 n
```

  1. ESTRUTURA E PROPÓSITO DA TABELA XREF

    A tabela xref contém a posição (offset em bytes) de cada objeto no arquivo. Sua estrutura consiste em:
    1. A palavra-chave xref em uma linha sozinha
    2. Uma linha com dois números: o primeiro objeto na seção e o número de entradas
    3. Uma entrada para cada objeto, cada uma com exatamente 20 bytes:
    - Offset de 10 dígitos (com zeros à esquerda)
    - Um espaço
    - Número de geração de 5 dígitos (geralmente 00000)
    - Um espaço
    - Um caractere 'n' (para objetos em uso) ou 'f' (para objetos livres)
    - Um espaço e uma quebra de linha
  2. Escreva o conteúdo do documento até antes da seção xref  
  - Inclua o cabeçalho (%SPDF-1.0)
  - Crie todos os objetos numerados
  3. Anote a posição (offset) do início de cada objeto  
  - A posição é o número de bytes desde o início do arquivo
  - Para o objeto 0 (especial), sempre use offset 0, geração 65535 e flag 'f'
  4. Crie a tabela xref usando os offsets anotados  
  Comece com a linha "xref"
  - Na próxima linha, coloque "0 N" onde N é o número total de objetos + 1 (inclui o objeto 0)
  - Para cada objeto, adicione uma entrada de 20 bytes
  5. Complete com a seção trailer  
  - Adicione a palavra-chave "trailer" seguida pelo dicionário trailer
  - Adicione "startxref" seguido pela posição da palavra-chave "xref"
  - Termine com "%%EOF"

8. **Trailer**: Informações finais do documento:
```
trailer
<< /Size 3 /Root 1 0 R >>
startxref
183
%%EOF
```

## Funcionalidades Requeridas do Programa

O programa deve implementar as seguintes funcionalidades:

1. **Validação Estrutural**:
  - Verificar se o cabeçalho SimplePDF está correto
  - Validar a sintaxe de cada objeto e suas propriedades
  - Verificar se todas as referências apontam para objetos existentes
  - Validar a estrutura hierárquica (catálogo → páginas → página)
  - Verificar consistência da tabela xref
  - Validar o trailer

2. **Extração de Informações**:  
  - Conteúdo textual de cada página
  - Metadados do documento (título, autor, data)
  - Estrutura de páginas (quantidade, tamanho)
  - Fontes utilizadas
  - Estatísticas de objetos por tipo

3. **Transformações**:
  - Extração de todo texto para formato TXT;
  - Geração de sumário baseado na estrutura
  - Listagem hierárquica de objetos e suas dependências
  - Conversão de coordenadas de texto para formato mais legível

4. **Análise Avançada**:
  - Detecção de objetos não referenciados
  - Identificação de ciclos de referência
  - Análise de eficiência de armazenamento
  - Sugestões de otimização da estrutura

5. **O programa deve ler dois arquivos**:
  - O documento SimplePDF a ser analisado
  - Um arquivo de configuração que define parâmetros de análise e transformação

## Arquivo de Configuração

O arquivo sera disponibilizado com o mesmo nome do arquivo pdf, porem com o formato .py

Teste01.pdf
```
%SPDF-1.0
...
```

Teste01.py
```
extrair_texto=True
gerar_sumario=True
detectar_ciclos=True
nivel_detalhe='Completo'
validar_xref=True
```

Opções:
- **extrair_texto** (bool): Habilita a extração de texto do documento.
- **gerar_sumario** (bool): Gera um sumário do conteúdo do documento.
- **detectar_ciclos** (bool): Detecta possíveis referências cíclicas no documento.
- **nivel_detalhe** (str): Define o nível de detalhe da análise ('Completo', 'Basico', 'Nulo').
- **validar_xref** (bool): Valida as referências cruzadas dentro do documento.

## Saída do Programa

O programa deve gerar um relatório com as seguintes seções:
```
VALIDAÇÃO:
[OK ou ERRO] Estrutura geral
[OK ou ERRO] Sintaxe de objetos
[OK ou ERRO] Referências
[OK ou ERRO] Tabela xref
[Detalhes de erros, se houver]

ESTATÍSTICAS:
Total de objetos: X
Objetos por tipo: Catalog=1, Pages=1, Page=Y, Font=Z, ...
Total de páginas: Y
Tamanho do documento: W bytes
Overhead estrutural: V bytes (P%)

CONTEÚDO:
Título: [título do documento]
Autor: [autor do documento]
Data de criação: [data]
Texto extraído: [primeiros 200 caracteres...]

ÁRVORE DE OBJETOS:
1: Catalog
  +- 2: Pages
    +- 3: Page
    +- 4: Font
    +- 5: Contents (stream)
6: Metadata

ANÁLISE AVANÇADA:
[Resultados de análises específicas definidas no arquivo de configuração]
```

## AS TRANSFORMAÇÕES ESPECIFICADAS

1. **Extração de texto para formato TXT**

 Esta transformação envolve **localizar e extrair todo o conteúdo textual do documento SimplePDF**, que está disperso em diferentes objetos de conteúdo dentro de streams e **convertê-lo para um arquivo de texto simples e contínuo**.

 Por exemplo: No **Exemplo 2**, o texto está dentro de **objetos stream** com comandos como **(Relatório Financeiro: Primeiro Trimestre) Tj**. O programa precisa **identificar esses comandos, extrair o texto entre parênteses**, e **organizá-lo em uma sequência lógica**, **respeitando a ordem das páginas e a posição dos elementos de texto**.

2. **Geração de sumário baseado na estrutura**

  Esta transformação requer **analisar a estrutura hierárquica do documento (especialmente objetos do tipo Outlines)** e **gerar uma representação textual organizada que funcione como um sumário ou índice**.

  Por exemplo: No **Exemplo 2, os objetos 11, 12 e 13 formam uma estrutura de outlines com títulos** como "Resumo Executivo", "Detalhamento de Vendas" e "Projeções Futuras". **O programa precisa reconhecer essa hierarquia (quem é pai de quem, quem vem antes ou depois)** e **organizá-la em um formato de sumário**.

3. **Listagem hierárquica de objetos e suas dependências**

  Esta transformação envolve **mapear todas as relações entre objetos (quem referência quem)** e **apresentá-las em um formato visual hierárquico, como uma árvore**.

  Por exemplo: O **objeto raiz (Catalog)** referência **Pages**, que por sua vez referência **Page**, que referência **Font e Contents**. Isso forma uma **estrutura em árvore que precisa ser extraída e apresentada de forma clara**, como mostrado na seção ÁRVORE DE OBJETOS da saída esperada.

4. **Conversão de coordenadas de texto para formato mais legível**

  Esta transformação requer **interpretar as coordenadas e comandos de posicionamento de texto (como 100 700 Td)** e **convertê-los para um formato que seja mais intuitivo para humanos**, possivelmente relacionando-os com a posição na página.

  Por exemplo: Em vez de mostrar **100 700 Td**, o programa poderia **converter** isso **para** **"Posição: 100px da esquerda, 92px do topo da página" (considerando que em PDF as coordenadas Y geralmente começam da parte inferior da página)**.

## Exemplos

### Exemplo 1

Exemplo 1
```
%SPDF-1.0

1 0 obj
<< /Type /Catalog
/Pages 2 0 R
/Metadata 7 0 R
/Outlines 8 0 R
>>
endobj

2 0 obj
<< /Type /Pages
/Kids [3 0 R 4 0 R]
/Count 2
>>
endobj

3 0 obj
<< /Type /Page
/Parent 2 0 R
/Resources << /Font << /F1 5 0 R >> >>
/MediaBox [0 0 612.0 792.0]
/CropBox [10.5 10.5 601.5 781.5]
/Rotate 0
/Contents 6 0 R
>>
endobj

4 0 obj
<< /Type /Page
/Parent 2 0 R
/Resources << /Font << /F1 5 0 R >> >>
/MediaBox [0 0 612.0 792.0]
/CropBox [10.5 10.5 601.5 781.5]
/Rotate 0
/Contents 9 0 R
>>
endobj

5 0 obj
<< /Type /Font
/Subtype /Type1
/BaseFont /Helvetica
/FontDescriptor 10 0 R
>>
endobj

6 0 obj
<< /Length 175 >>
stream
BT
/F1 24 Tf
100 700 Td
(Relatório Financeiro: Primeiro Trimestre) Tj
/F1 12 Tf
0 -50 Td
(Total de vendas: 1423 unidades) Tj
0 -20 Td
(Receita total: 158432.75 reais) Tj
ET
endstream
endobj

7 0 obj
<< /Title (Relatório Financeiro Trimestral)
/Author (Departamento Financeiro)
/CreationDate (D:20230418090000)
/Keywords (finanças, relatório, trimestral)
/Version 1.2
/PageCount 2
>>
endobj

8 0 obj
<< /Type /Outlines
/Count 3
/First 11 0 R
/Last 13 0 R
>>
endobj

9 0 obj
<< /Length 322 >>
stream
BT
/F1 18 Tf
100 700 Td
(Detalhamento por Categoria) Tj
/F1 12 Tf
0 -40 Td
(Produto A: 523 unidades, valor unitário 89.99, total 47054.77) Tj
0 -20 Td
(Produto B: 347 unidades, valor unitário 125.50, total 43548.50) Tj
0 -20 Td
(Produto C: 553 unidades, valor unitário 122.65, total 67825.45) Tj
0 -40 Td
(Média de vendas por produto: 474.33 unidades) Tj
ET
endstream
endobj

10 0 obj
<< /Type /FontDescriptor
/FontName /Helvetica
/Flags 32
/FontBBox [-166 -225 1000 931]
/ItalicAngle 0
/Ascent 718
/Descent -207
/CapHeight 718
/StemV 88
/MissingWidth 278
>>
endobj

11 0 obj
<< /Title (Resumo Executivo)
/Parent 8 0 R
/Next 12 0 R
/Dest [3 0 R /FitH 792.0]
>>
endobj

12 0 obj
<< /Title (Detalhamento de Vendas)
/Parent 8 0 R
/Prev 11 0 R
/Next 13 0 R
/Dest [4 0 R /FitH 792.0]
>>
endobj

13 0 obj
<< /Title (Projeções Futuras)
/Parent 8 0 R
/Prev 12 0 R
/Dest [4 0 R /XYZ 100 450 1.25]
>>
endobj

14 0 obj
<< /Type /Metadata
/Subtype /XML
/Length 340
>>
stream
<?xpacket begin='' id='W5M0MpCehiHzreSzNTczkc9d'?>
<x:xmpmeta xmlns:x='adobe:ns:meta/'>
<rdf:RDF xmlns:rdf='http://www.w3.org/1999/02/22-rdf-syntax-ns#'>
<rdf:Description rdf:about=''
xmlns:pdf='http://ns.adobe.com/pdf/1.3/'>
<pdf:Producer>SimplePDF Generator 1.0</pdf:Producer>
</rdf:Description>
</rdf:RDF>
</x:xmpmeta>
<?xpacket end='w'?>
endstream
endobj
xref
0 15
0000000000 65535 f
0000000010 00000 n
0000000099 00000 n
0000000163 00000 n
0000000344 00000 n
0000000525 00000 n
0000000624 00000 n
0000000852 00000 n
0000001048 00000 n
0000001126 00000 n
0000001500 00000 n
0000001712 00000 n
0000001818 00000 n
0000001940 00000 n
0000002052 00000 n
trailer
<< /Size 15
/Root 1 0 R
/Info 7 0 R
>>
startxref
2500
%%EOF
```

### Exemplo 2

Exemplo 2
```
%SPDF-1.0

1 0 obj
<< /Type /Catalog
/Pages 2 0 R
/Metadata 7 0 R
/Outlines 8 0 R
>>
endobj

2 0 obj
<< /Type /Pages
/Kids [3 0 R 4 0 R]
/Count 2
>>
endobj

3 0 obj
<< /Type /Page
/Parent 2 0 R
/Resources << /Font << /F1 5 0 R >> >>
/MediaBox [0 0 612.0 792.0]
/CropBox [10.5 10.5 601.5 781.5]
/Rotate 0
/Contents 6 0 R
>>
endobj

4 0 obj
<< /Type /Page
/Parent 2 0 R
/Resources << /Font << /F1 5 0 R >> >>
/MediaBox [0 0 612.0 792.0]
/CropBox [10.5 10.5 601.5 781.5]
/Rotate 0
/Contents 9 0 R
>>
endobj

5 0 obj
<< /Type /Font
/Subtype /Type1
/BaseFont /Helvetica
/FontDescriptor 10 0 R
>>
endobj

6 0 obj
<< /Length 175 >>
stream
BT
/F1 24 Tf
100 700 Td
(Relatório Financeiro: Primeiro Trimestre) Tj
/F1 12 Tf
0 -50 Td
(Total de vendas: 1423 unidades) Tj
0 -20 Td
(Receita total: 158432.75 reais) Tj
ET
endstream
endobj

7 0 obj
<< /Title (Relatório Financeiro Trimestral)
/Author (Departamento Financeiro)
/CreationDate (D:20230418090000)
/Keywords (finanças, relatório, trimestral)
/Version 1.2
/PageCount 2
>>
endobj

8 0 obj
<< /Type /Outlines
/Count 3
/First 11 0 R
/Last 13 0 R
>>
endobj

9 0 obj
<< /Length 322 >>
stream
BT
/F1 18 Tf
100 700 Td
(Detalhamento por Categoria) Tj
/F1 12 Tf
0 -40 Td
(Produto A: 523 unidades, valor unitário 89.99, total 47054.77) Tj
0 -20 Td
(Produto B: 347 unidades, valor unitário 125.50, total 43548.50) Tj
0 -20 Td
(Produto C: 553 unidades, valor unitário 122.65, total 67825.45) Tj
0 -40 Td
(Média de vendas por produto: 474.33 unidades) Tj
ET
endstream
endobj

10 0 obj
<< /Type /FontDescriptor
/FontName /Helvetica
/Flags 32
/FontBBox [-166 -225 1000 931]
/ItalicAngle 0
/Ascent 718
/Descent -207
/CapHeight 718
/StemV 88
/MissingWidth 278
>>
endobj

11 0 obj
<< /Title (Resumo Executivo)
/Parent 8 0 R
/Next 12 0 R
/Dest [3 0 R /FitH 792.0]
>>
endobj

12 0 obj
<< /Title (Detalhamento de Vendas)
/Parent 8 0 R
/Prev 11 0 R
/Next 13 0 R
/Dest [4 0 R /FitH 792.0]
>>
endobj

13 0 obj
<< /Title (Projeções Futuras)
/Parent 8 0 R
/Prev 12 0 R
/Dest [4 0 R /XYZ 100 450 1.25]
>>
endobj

14 0 obj
<< /Type /Metadata
/Subtype /XML
/Length 340
>>
stream
<?xpacket begin='' id='W5M0MpCehiHzreSzNTczkc9d'?>
<x:xmpmeta xmlns:x='adobe:ns:meta/'>
<rdf:RDF xmlns:rdf='http://www.w3.org/1999/02/22-rdf-syntax-ns#'>
<rdf:Description rdf:about=''
xmlns:pdf='http://ns.adobe.com/pdf/1.3/'>
<pdf:Producer>SimplePDF Generator 1.0</pdf:Producer>
</rdf:Description>
</rdf:RDF>
</x:xmpmeta>
<?xpacket end='w'?>
endstream
endobj

xref
0 15
0000000000 65535 f
0000000010 00000 n
0000000099 00000 n
0000000163 00000 n
0000000344 00000 n
0000000525 00000 n
0000000624 00000 n
0000000852 00000 n
0000001048 00000 n
0000001126 00000 n
0000001500 00000 n
0000001712 00000 n
0000001818 00000 n
0000001940 00000 n
0000002052 00000 n

trailer
<< /Size 15
/Root 1 0 R
/Info 7 0 R
>>
startxref
2500

%%EOF
```

## Dicas

1. **Comece simples**: Crie primeiro documentos com poucos objetos e estrutura simples.
2. **Cálculo manual de offsets**: Para documentos simples, conte manualmente os bytes para cada
posição. Lembre-se de que cada caractere (incluindo espaços e quebras de linha) conta como
um byte.
3. **Uso de ferramentas**:  
  - Os alunos podem escrever um pequeno script auxiliar que conte o número de bytes
até cada objeto
  - Editores hexadecimais podem ser úteis para verificar posições
4. **Construção gradual**:  
  - Comece com o cabeçalho e objetos simples
  - Adicione um marcador temporário para a tabela xref (ex: "XREF_AQUI")
  - Conte a posição de cada objeto
  - Substitua o marcador pela tabela xref real
  - Adicione a seção trailer
5. **Formato de quebra de linha**: Certifique-se de usar consistentemente o mesmo formato de
quebra de linha (CRLF ou LF) ao contar bytes.
6. **Verificação do documento**:  
Após criar o documento, percorra-o byte a byte para verificar se os offsets na tabela xref apontam corretamente para o início de cada objeto

**CASOS DE TESTE IMPORTANTES**

Os alunos devem criar documentos de teste que cubram:
1. **Estrutura básica**: Documento mínimo com catálogo e páginas
2. **Reais vs. Inteiros**: Uso de números inteiros e de ponto flutuante
3. **Tipos de dados**: Todas as variações (strings, nomes, arrays, dicionários)
4. **Referências circulares**: Objetos que se referenciam mutuamente
5. **Conteúdo de texto**: Objetos stream com comandos de texto
6. **Estrutura hierárquica complexa**: Vários níveis de referência entre objetos
7. **Metadados completos**: Informações detalhadas no objeto de metadados
Criando arquivos de teste apropriados, os alunos poderão verificar se suas implementações lidam
corretamente com todos os aspectos do formato SimplePDF, especialmente a interpretação da
tabela xref, que é crucial para a navegação eficiente dentro do documento.

# Código

In [182]:
# Downloading pdfs and configs (at least 3 examples)
!wget -nc https://raw.githubusercontent.com/JoaoVLima/ActualSimplePDFValidator/main/exemplos/exemplo3.pdf
!wget -nc https://raw.githubusercontent.com/JoaoVLima/ActualSimplePDFValidator/main/exemplos/exemplosimples.py

File ‘exemplo3.pdf’ already there; not retrieving.

File ‘exemplosimples.py’ already there; not retrieving.



**Config**

In [183]:
extrair_texto=None
gerar_sumario=None
detectar_ciclos=None
nivel_detalhe=None
validar_xref=None

In [184]:
from exemplosimples import *

In [185]:
print(extrair_texto,
      gerar_sumario,
      detectar_ciclos,
      nivel_detalhe,
      validar_xref)

True True True Completo True


**Read File**

In [186]:
def read_pdf(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        text = file.read()
    return text

In [187]:
FILE = read_pdf('exemplo3.pdf')

In [188]:
FILE

'%SPDF-1.0\n1 0 obj\n<<\n/Type /Catalog\n/Pages 3 0 R\n/Outlines 2 0 R\n>>\nendobj\n2 0 obj\n<<\n/Type /Outlines\n/Count 0\n>>\nendobj\n3 0 obj\n<<\n/Type /Pages\n/Count 1\n/Kids [ 4 0 R ]\n>>\nendobj\n4 0 obj\n<<\n/Type /Page\n/Parent 3 0 R\n/Resources << /ProcSet 6 0 R >>\n/MediaBox [ 0 0 612 792 ]\n/Contents 5 0 R\n>>\nendobj\n5 0 obj\n<<\n/Length 35\n>>\nstream\n%place page marking operators here\nendstream\nendobj\n6 0 obj\n[ /PDF ]\nendobj\nxref\n0 7\n0000000000 65535 f\n0000000010 00000 n\n0000000075 00000 n\n0000000121 00000 n\n0000000180 00000 n\n0000000301 00000 n\n0000000385 00000 n\ntrailer\n<<\n/Size 7\n/Root 1 0 R\n>>\nstartxref\n409\n%%EOF'

In [189]:
FILE_SIZE = len(FILE)
FILE_SIZE

604

In [190]:
FILE = FILE.split('\n') # \n counts as 1

In [191]:
FILE[0:10]

['%SPDF-1.0',
 '1 0 obj',
 '<<',
 '/Type /Catalog',
 '/Pages 3 0 R',
 '/Outlines 2 0 R',
 '>>',
 'endobj',
 '2 0 obj',
 '<<']

In [192]:
FILE[-20:]

['6 0 obj',
 '[ /PDF ]',
 'endobj',
 'xref',
 '0 7',
 '0000000000 65535 f',
 '0000000010 00000 n',
 '0000000075 00000 n',
 '0000000121 00000 n',
 '0000000180 00000 n',
 '0000000301 00000 n',
 '0000000385 00000 n',
 'trailer',
 '<<',
 '/Size 7',
 '/Root 1 0 R',
 '>>',
 'startxref',
 '409',
 '%%EOF']

In [193]:
LINE_SIZE = [len(line)+1 for line in FILE]
LINE_SIZE[0:10]

[10, 8, 3, 15, 13, 16, 3, 7, 8, 3]

**Get Comments (header and eof)**

In [194]:
def get_comment(line):
    return line[line.index('%'):] if '%' in line else None

In [195]:
def ignore_comment(line):
    return line[:line.index('%')] if '%' in line else line

In [196]:
def get_comments(file):
    comments = []
    for line in file:
        if '%' in line:
            comments.append(get_comment(line))
    return comments

In [197]:
COMMENTS = get_comments(FILE)
COMMENTS[0:10]

['%SPDF-1.0', '%place page marking operators here', '%%EOF']

In [198]:
HEADER = COMMENTS.pop(0) # First line that starts with %
HEADER

'%SPDF-1.0'

In [199]:
EOF = COMMENTS.pop(-1) # Last line that starts with %
EOF

'%%EOF'

In [200]:
COMMENTS[0:10]

['%place page marking operators here']

**Check Header and Footer**

In [201]:
def check_header(header):
    return header.startswith('%SPDF-') or header.startswith('%PDF-')

In [202]:
check_header(HEADER)

True

In [203]:
def check_eof(eof):
    return eof == '%%EOF'

In [204]:
check_eof(EOF)

True

**Remove Comments**

In [205]:
def remove_comments(file):
    return [ignore_comment(line) for line in file]

In [206]:
FILE = remove_comments(FILE)

**Remove Whitespace**

In [207]:
def remove_whitespace(file):
    return [line.strip() for line in file]

In [208]:
FILE = remove_whitespace(FILE)

**Get Objects**

In [209]:
def object_dict(line_index=None, xref_address=None, obj_id=None, obj_gen=None, content=None):
  return dict(
      line_index=line_index,
      xref_address=xref_address,
      obj_id=obj_id,
      obj_gen=obj_gen,
      content=content
  )

In [210]:
def get_objects(file):
    objects = []
    object = {}
    content = []
    get_content = False

    for i, line in enumerate(file):
        line = ignore_comment(line)
        parts = line.split()
        if len(parts) == 3 and parts[2] == 'obj':  # line that starts an object
            object = object_dict(i, sum(LINE_SIZE[0:i]), parts[0], parts[1])
            content = []  # Reset content for new object
            get_content = True
        elif get_content:
            if len(parts) == 1 and parts[0] == 'endobj':  # line that ends the object
                object['content'] = content
                objects.append(object)
                object = {}
                content = []
                get_content = False
            else:
                content.append(line)

    return objects

In [211]:
OBJECTS = get_objects(FILE)

In [212]:
OBJECTS[0:5]

[{'line_index': 1,
  'xref_address': 10,
  'obj_id': '1',
  'obj_gen': '0',
  'content': ['<<',
   '/Type /Catalog',
   '/Pages 3 0 R',
   '/Outlines 2 0 R',
   '>>']},
 {'line_index': 8,
  'xref_address': 75,
  'obj_id': '2',
  'obj_gen': '0',
  'content': ['<<', '/Type /Outlines', '/Count 0', '>>']},
 {'line_index': 14,
  'xref_address': 121,
  'obj_id': '3',
  'obj_gen': '0',
  'content': ['<<', '/Type /Pages', '/Count 1', '/Kids [ 4 0 R ]', '>>']},
 {'line_index': 21,
  'xref_address': 180,
  'obj_id': '4',
  'obj_gen': '0',
  'content': ['<<',
   '/Type /Page',
   '/Parent 3 0 R',
   '/Resources << /ProcSet 6 0 R >>',
   '/MediaBox [ 0 0 612 792 ]',
   '/Contents 5 0 R',
   '>>']},
 {'line_index': 30,
  'xref_address': 301,
  'obj_id': '5',
  'obj_gen': '0',
  'content': ['<<', '/Length 35', '>>', 'stream', '', 'endstream']}]

In [213]:
def content_dict(content):
    dictionary = {}
    for line in content:
        parts = line.split()
        if '<<' in parts:
            parts.remove('<<')
        if '>>' in parts:
            parts.remove('>>')
        if len(parts) < 2:
            continue
        key = parts[0]
        value = parts[1:]
        dictionary[key] = value
    return dictionary

In [214]:
for obj in OBJECTS:
    obj['content'] = content_dict(obj['content'])

In [215]:
OBJECTS[0:5]

[{'line_index': 1,
  'xref_address': 10,
  'obj_id': '1',
  'obj_gen': '0',
  'content': {'/Type': ['/Catalog'],
   '/Pages': ['3', '0', 'R'],
   '/Outlines': ['2', '0', 'R']}},
 {'line_index': 8,
  'xref_address': 75,
  'obj_id': '2',
  'obj_gen': '0',
  'content': {'/Type': ['/Outlines'], '/Count': ['0']}},
 {'line_index': 14,
  'xref_address': 121,
  'obj_id': '3',
  'obj_gen': '0',
  'content': {'/Type': ['/Pages'],
   '/Count': ['1'],
   '/Kids': ['[', '4', '0', 'R', ']']}},
 {'line_index': 21,
  'xref_address': 180,
  'obj_id': '4',
  'obj_gen': '0',
  'content': {'/Type': ['/Page'],
   '/Parent': ['3', '0', 'R'],
   '/Resources': ['/ProcSet', '6', '0', 'R'],
   '/MediaBox': ['[', '0', '0', '612', '792', ']'],
   '/Contents': ['5', '0', 'R']}},
 {'line_index': 30,
  'xref_address': 301,
  'obj_id': '5',
  'obj_gen': '0',
  'content': {'/Length': ['35']}}]

**Get xref**

In [216]:
def get_xref_address(file):
    for i, line in enumerate(file):
        if line == 'xref':
            return sum(LINE_SIZE[0:i])
    return -1

In [217]:
XREF_ADDRESS = get_xref_address(FILE)
XREF_ADDRESS

409

In [218]:
def get_xref_line(file):
    for i, line in enumerate(file):
        if line == 'xref':
            return i
    return -1

In [219]:
XREF_LINE = get_xref_line(FILE)
XREF_LINE

41

**Check xref**

In [220]:
def check_xref_address(file):
    for i, line in enumerate(file):
        if line == 'startxref':
            return int(file[i+1]) == XREF_ADDRESS
    return False

In [221]:
check_xref_address(FILE)

True

In [222]:
def check_xref(file):
    if file[XREF_LINE+1].split()[1] != str(len(OBJECTS)+1):
        return False

    if file[XREF_LINE+2] != '0000000000 65535 f':
        return False

    for i, line in enumerate(file[XREF_LINE+3:]):
        if line in ('',' ','trailer'):
            break

        parts = line.split()
        if len(parts) != 3:
            return False

        if OBJECTS[i]['xref_address'] != int(parts[0]):
            return False

    return True

In [223]:
check_xref(FILE)

True

**Get Trailer**

In [234]:
def get_trailer_line(file):
    trailer_index = -1
    for i in range(len(file)-1,-1,-1):
        if file[i] == 'trailer':
            trailer_index = i
            break

    return trailer_index

In [235]:
TRAILER_LINE = get_trailer_line(FILE)

**Check Trailer**

In [226]:
def check_trailer(file):
    trailer = file[TRAILER_LINE:]
    print(trailer)

In [227]:
check_trailer(FILE)

['trailer', '<<', '/Size 7', '/Root 1 0 R', '>>', 'startxref', '409', '']


In [228]:
def dict_trailer(file):
    trailer = file[TRAILER_LINE:]
    return content_dict(trailer[2:-3])

In [229]:
dict_trailer(FILE)

{'/Size': ['7'], '/Root': ['1', '0', 'R']}